In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import pca_pseudo_grid_search

#otros
import os
import numpy as np
import pandas as pd
import timeit
from sklearn.externals import joblib

tipo = pruebas[3]
dataset_name = datasets[1]
dims = dimensions[dataset_name]

Using Theano backend.
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/francisco/miniconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Pruebas con el dataset de Twitter

In [3]:
print tipo
print dataset_name
print dims
print data_path

pca
twitter
2000
data


In [4]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [5]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print "Todos los datos disponibles obtenidos"

Todos los datos disponibles obtenidos


```python

saved_paths = [ruta_1,...,ruta_n]
```

In [6]:
t_total_i = timeit.default_timer()

pca_paths = os.path.join(models_path, tipo, "%s_paths.pkl" % dataset_name)

if os.path.exists(pca_paths):
    print "Cargando rutas de modelos adaptados."
    saved_paths = joblib.load(pca_paths)
else:
    print "Creando modelos de adaptacion..."

    n_components = [int(dims/2), int(dims/3), int(dims/4)]

    parametros = {
        'n_components': n_components
    }


    saved_paths = pca_pseudo_grid_search(X, parametros, models_path, tipo, dataset_name)

    print "Modelos creados.\n"
    
    joblib.dump(saved_paths, pca_paths)
    print "Guardando rutas en %s" % pca_paths
    
t_total_f = timeit.default_timer()

t_total = (t_total_f - t_total_i)/60.

print "Creacion de modelos realizada en %.2fm" % t_total

Cargando rutas de modelos adaptados.
Creacion de modelos realizada en 0.00m


In [7]:
saved_paths

['models/pca/twitter_0.pkl',
 'models/pca/twitter_1.pkl',
 'models/pca/twitter_2.pkl']

```python
domain_msda_paths = [ruta_1,...,ruta_n]
```

In [8]:
best_pca_paths = os.path.join(models_path, tipo, "best_%s_paths.pkl" % dataset_name )

if os.path.exists(best_pca_paths):
    #se cargan las rutas de los modelos guardados
    domain_pca_paths = joblib.load(best_pca_paths)   

else:
    domain_pca_paths = []
    #se obtienen los mejores modelos 
    for domain in domains:
        # se obtienen los datos del dominio
        X_tr = np.asarray(labeled[domain]['X_tr'].todense())
        y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)
        
        
        
        print "Obteniendo mejor clasificador para %s" % domain
        
        new_clf = get_best_score(X_tr, y_tr, clasifier='SVC', n_jobs=4)
        new_score = new_clf.best_score_
        
        print "Sin adaptar: %.4f" % (new_score)
        
        i = 1

        best_score = 0
        best_model = None
        for pca_model_path in saved_paths:
            
            pca_model = joblib.load(pca_model_path)
            n_components = pca_model.n_components_
            
            print "%d) n_components=%d " % (i, n_components),

            # se adaptan los datos
            X_deep = pca_model.transform(X_tr)

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, clasifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_model = pca_model

            i = i+1

        #se guarda el mejor modelo para este dominio
        print "\n\tMejor modelo: n_components=%d" % (best_model.n_components_)
        best_pca_path = os.path.join(models_path, tipo, "%s_%s.pkl" % (dataset_name, domain))
        print "\tGuardando en %s\n" % best_pca_path
        joblib.dump(best_model, best_pca_path)
        
        domain_pca_paths.append(best_pca_path)
    
    print "Guardando rutas en %s" % best_pca_paths
    joblib.dump(domain_pca_paths, best_pca_paths)

print "\nObtencion de los mejores modelos terminada."


Obtencion de los mejores modelos terminada.


In [9]:
domain_pca_paths

['models/pca/twitter_rio2016.pkl', 'models/pca/twitter_thevoice.pkl']

In [10]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_pca_path = os.path.join(models_path, tipo, "%s_%s.pkl" % (dataset_name, domain))
    print best_pca_path
    
    if os.path.exists(best_pca_path):
        print "Cargando mejor modelo para %s" % domain
        pca_model = joblib.load(best_pca_path)
        
        print "\tAdaptando dominio"
        X_tr = np.asarray(labeled[domain]['X_tr'].todense())
        X_ts = np.asarray(labeled[domain]['X_ts'].todense())
        
        
        tr_reps = pca_model.transform(X_tr)
        ts_reps = pca_model.transform(X_ts)

        adapted[domain] = {
            'X_tr': tr_reps,
        }
        
        i = i+1
    else:
        print "\tGenerar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

models/pca/twitter_rio2016.pkl
Cargando mejor modelo para rio2016
	Adaptando dominio
models/pca/twitter_thevoice.pkl
Cargando mejor modelo para thevoice
	Adaptando dominio
2/2 dominios adaptados


In [11]:
%%time

df = pd.DataFrame(columns=dataframe_columns)

i=0
# por cada par posible para adaptar
for src in domains:
    best_pca_path = os.path.join(models_path, tipo, "%s_%s.pkl" % (dataset_name, src))
    adapter = joblib.load(best_pca_path)
    
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 2" % (i+1)
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            #model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            #svc = get_best_score(X_tr, y_tr)
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = adapter.transform(X_ts)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            print "Entrenando clasificador adaptado."
            svc_a = get_best_score(X_tr_a, y_tr, clasifier='SVC', n_jobs=4)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['PCA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i+=1

print "\nPruebas completadas"

Tarea 1 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.
Tarea 2 de 2
Cargando modelo existente.
Entrenando clasificador adaptado.

Pruebas completadas
CPU times: user 1.65 s, sys: 68 ms, total: 1.72 s
Wall time: 5.48 s


In [12]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,PCA,r->t,rio2016,thevoice,16.153846,23.076923,6.923077
1,PCA,t->r,thevoice,rio2016,19.791667,21.875000,2.083333


In [13]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")
print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/pca/twitter.csv
Resultados guardados.
